In [1]:
import requests
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
import json
import time
import time
import random
import datetime
#time.sleep(random.uniform(1, 5))

### Отзывы о банке

In [ ]:
import numpy as np
import datetime
import time
import random

# Минимальная дата, с которой интересуют отзывы
until_date = '2019-04-30'

bankiru_comments_all = pd.DataFrame(columns = ['topic', 'time', 'date', 'evaluation', 'rating', 'comment'])

def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text)) 

page = 1

earliest_date = datetime.datetime.now().date()

while earliest_date >= pd.to_datetime(until_date).date():
    
#    print(page)
#    time.sleep(1.23)
    main_url = 'https://www.banki.ru/services/responses/bank/tcs/?page={}'
    text_page = requests.get(main_url.format(page))
    comment_page = text_page.content
    comment_page = BeautifulSoup(comment_page, "lxml")

    # Тема обращения
    topics = []
    for cont in comment_page.findAll('a', attrs = {'class':"header-h3",'data-test':"responses-header"}):
        name = re.split('No.',html_stripper(cont))
        topics += name

    # Наличие оценки
    evaluation = []    
    for cont in comment_page.findAll('span', attrs = {'class':"text-label",'data-test':"responses-estimate"}):
        e = re.sub(':', '', html_stripper(cont))
        evaluation += [e]

    # Оценка пользователя
    ratings = []
    for cont in comment_page.findAll('span', attrs = {'data-test':"responses-rating-grade"}):
        n = re.sub('\t|\n', '', html_stripper(cont)) 
        ratings += [n]
    ratings = list(map(int,ratings))

    # Добавляем 0 вместо пустых рейтингов
    if sum(np.array(evaluation)=="Без оценки")>0:
        n_missing = np.where(np.array(evaluation)=="Без оценки")[0][0]
        for n_missing in np.where(np.array(evaluation)=="Без оценки")[0]:
            ratings = ratings[:(n_missing)] + [0] + ratings[(n_missing):]

    # Время обращения
    times = []
    for cont in comment_page.findAll('time', attrs = {'class':"display-inline-block",'data-test':"responses-datetime"}):
        time = re.split('No.', html_stripper(cont))
        times += time
    dates = [d.date() for d in pd.to_datetime(times)]
    earliest_date = dates[-1]

    # Текст обращения
    texts = []
    for cont in comment_page.findAll('div', attrs = {'class':"responses__item__message markup-inside-small markup-inside-small--bullet",'data-full':""}):
        t = html_stripper(cont)
        texts += [re.sub(r'\t|\n|\xa0', '', t)]

    bankiru_comments = pd.DataFrame({"topic": topics,
                                     "time": times,
                                     "date": dates,
                                     "evaluation": evaluation, 
                                     "rating": ratings,
                                     "comment": texts})    
    bankiru_comments_all = pd.concat([bankiru_comments_all, bankiru_comments], axis = 0)
    page += 1
bankiru_comments_all.to_excel("comments_tcs.xlsx", index = False, encoding = 'utf-8')

In [3]:
bankiru_comments_all

,topic,time,date,evaluation,rating,comment
0,Очень разочаровали: подключили платные услуги ...,14.06.2020 17:06,2020-06-14,Оценка,2,Приходит мне сегодня в обед чудесное смс от Ти...
1,Спасибо огромное сотрудникам банка,14.06.2020 15:41,2020-06-14,Оценка,5,Хочу поблагодарить сотрудников банка Тинькофф....
2,Нет начисления рокетрублей по акции,14.06.2020 15:32,2020-06-14,Оценка,1,"После изменения условий в ""Рокетбанке"" Тинькоф..."
3,Супер клиентоориентированный банк!,14.06.2020 14:57,2020-06-14,Оценка,5,"Давно уже являюсь клиентом данного банка, в пр..."
4,"Яндекс плюс Тинькофф — больше выгоды, но не дл...",14.06.2020 13:29,2020-06-14,Оценка,1,"Уважаемые представители Тинькофф, ваши кредитн..."
...,...,...,...,...,...,...
20,Отзыв,04.12.2019 15:34,2019-04-12,Оценка,5,Хочу сказать спасибо огромное представителю ба...
21,Не приглашайте друзей,04.12.2019 15:24,2019-04-12,Оценка,1,"История такая , приглошал постоянно друзей в б..."
22,Благодарность,04.12.2019 15:12,2019-04-12,Оценка,5,"Шикарный банк, с выгодными условиями, комиссия..."
23,5+,04.12.2019 14:20,2019-04-12,Оценка,5,"Добрый день.Спасибо вам. Отличный банк, шикарн..."
